In [1]:
#The torch.nn namespace provides all the building blocks you need to build your own neural network.
#Every module in PyTorch subclasses the nn.Module. 
#module can consists other odels(layers)
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [3]:
#define neural network by subclassing nn.Module
#initialize network layers in __init__
#implements the operations on input data in the forward method
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
#Create instance of NeuralNetwork
#and move it to device
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
#Use model need us pass it the input data
#we can use by model(X),it will execute "forward"
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='cuda:0')


In [6]:
#model layers

#what happened when data go though layers
#input layer
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
#flatten layer

#to convert each 2D 28x28 image 
#into a contiguous array of 784 pixel values
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
#linear layer

# applies a linear transformation on the input 
# using its stored weights and biases.
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
#ReLU layer

#create the complex mappings 
#between the model’s inputs and outputs
#use nn.ReLU between our linear layers

print(f"Before ReLU:\n {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU:\n {hidden1}")

Before ReLU:
 tensor([[0.0000, 0.1668, 0.3100, 0.0000, 0.1796, 0.0000, 0.0000, 0.5117, 0.0000,
         0.1565, 0.1922, 0.0000, 0.0000, 0.1696, 0.1161, 0.0000, 0.0000, 0.1917,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.2069, 0.0000, 0.2154, 0.0000, 0.0000, 0.0438, 0.0023,
         0.0496, 0.0000, 0.0000, 0.0000, 0.0038, 0.4332, 0.0000, 0.0000, 0.1740,
         0.0000, 0.0000],
        [0.0000, 0.3394, 0.3167, 0.0000, 0.0000, 0.3622, 0.0000, 0.1876, 0.0000,
         0.1664, 0.0243, 0.0000, 0.0000, 0.1155, 0.1046, 0.0000, 0.0492, 0.3180,
         0.0000, 0.0000]], grad_fn=<ReluBackward0>)


After ReLU:
 tensor([[0.0000, 0.1668, 0.3100, 0.0000, 0.1796, 0.0000, 0.0000, 0.5117, 0.0000,
         0.1565, 0.1922, 0.0000, 0.0000, 0.1696, 0.1161, 0.0000, 0.0000, 0.1917,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.2069, 0.0000, 0.2154, 0.0000, 0.0000, 0.0438, 0.0023,
         0.0496, 0.0000, 0.0000, 0.0000, 0.0038, 0.4332, 0.0000, 0.0000, 0.1740,
         0.0000, 0.0000],
     

In [11]:
#sequential

#an ordered container of modules
#data is passed through all the modules in the same order as defined.
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [12]:
#softmax
#are used to scale to values [0, 1]
#dim indicates thedimension 
#along which the values must sum to 1
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [15]:
# Model Parameters
#layers are parameterized
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | \n\tValues : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | 
	Values : tensor([[ 0.0160, -0.0235, -0.0193,  ..., -0.0294, -0.0174, -0.0210],
        [-0.0202, -0.0017,  0.0187,  ...,  0.0289, -0.0142, -0.0344]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | 
	Values : tensor([-0.0151, -0.0023], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | 
	Values : tensor([[-0.0032, -0.0371, -0.0339,  ..., -0.0265,  0.0243,  0.0153],
        [ 0.0413, -0.0078, -0.0186,  ..., -0.0249, -0.0383, -0.0139]],
       device='cuda:0', gra